In [1]:
# First load in modules
import numpy as np
from math import *
import sys
import os
from distutils.spawn import find_executable  # This is to find Sextractor
import pickle                                # This is to save output files
import comest                                # This is to import ComEst

#####
#
# Set the file names and configurations
#
#####

# get the current working directory
CWD             =       os.path.dirname( os.path.abspath(__name__) )

# get the sex path
sex_exec        =       find_executable("sex")

# get the file name
# input image and weight map
path2img        =       os.path.join( CWD, "DEEPi.cutout.fits" )
path2wgt        =       os.path.join( CWD, "DEEPi.cutout.wgt.fits" )
# the directory to store output files (it will be created if it does not exist)
path2outdir     =       os.path.join( CWD, "outdir_example" )
# Sextractor configure files and output parameters. You can pass your own.
sex_config      =       os.path.join( os.path.dirname( comest.__file__ ), "templates/sex.config" )
sex_params      =       os.path.join( os.path.dirname( comest.__file__ ), "templates/sex.params" )
# the name root for the full run. You can name it differently.
full_root_name  =       "full"
# the name root for the BnB run. You can name it differently.
bnb_root_name   =       "bnb"
# Sextractor arguments
full_sex_args   =        "-DETECT_MINAREA 5   -DETECT_THRESH 1.5 -ANALYSIS_THRESH 1.5 -DEBLEND_NTHRESH 32 -DEBLEND_MINCONT 0.005 -CLEAN Y -CLEAN_PARAM 1.0 -BACKPHOTO_THICK 24.0 -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_IMAGE" + " " + path2wgt
bnb_sex_args    =        "-DETECT_MINAREA 350 -DETECT_THRESH 1.5 -ANALYSIS_THRESH 1.5 -DEBLEND_NTHRESH 32 -DEBLEND_MINCONT 0.005 -CLEAN Y -CLEAN_PARAM 1.0 -BACKPHOTO_THICK 24.0 -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_IMAGE" + " " + path2wgt

# image properties
img_zp          =       31.7    # the zeropoint
img_pixel_scale =       0.26    # image pixel arcsec/pixel
img_fwhm        =       0.9     # from Desai+12 i band image.

MAG_LO          =       20.0    # simulated magnitude range
MAG_HI          =       25.0    # simulated magnitude range

nsimimages      =       5      # the number of simulated images per set
ncpu            =       3      # the number of cpu cores used in simulation
nset            =       2      # the number of the sets, each set should contain nsimimages simulated images

# decide which mode to use for simulation
# here, I use bulgal, reagal, and pntsrc.
use_modgal      =       False    # galaxies distribution modelled from SE catalog - This is testing, and we do not suggest to use this. Better to remain it to be `False`.
use_bulgal      =       True     # bul + disk galaxies simulation
use_reagal      =       True     # galaxies draw from COSMOS
use_pntsrc      =       True     # point source simulation

# set the random seed
np.random.seed(232)

# Declare an output dict to store stuffs
HIST_DICT       =       {}

In [2]:
#####
#
# Now load the image
#
#####

# create an instance for image
fits_image_example  =   comest.ComEst.fitsimage(
                        path2img = path2img,
                        path2outdir = path2outdir,
                        sex_exec = sex_exec,
                        sex_config = sex_config,
                        sex_params = sex_params,
                        full_root_name = full_root_name,
                        bnb_root_name = bnb_root_name,
                        full_sex_args = full_sex_args,
                        bnb_sex_args = bnb_sex_args,
                        img_zp = img_zp,
                        img_pixel_scale = img_pixel_scale,
                        img_fwhm = img_fwhm)

# introduce yourself
fits_image_example.i_am()


# fits image information as below:
# path2img: /Users/inchiu/GitHub/ComEst/example/DEEPi.cutout.fits
# path2outdir: /Users/inchiu/GitHub/ComEst/example/outdir_example
# sex_exec: /usr/local/bin/sex
# sex_full_config: /usr/local/lib/python2.7/site-packages/comest/templates/sex.config
# sex_full_params: /usr/local/lib/python2.7/site-packages/comest/templates/sex.params
# full_root_name: full
# bnb_root_name: bnb
# full_sex_args: -DETECT_MINAREA 5   -DETECT_THRESH 1.5 -ANALYSIS_THRESH 1.5 -DEBLEND_NTHRESH 32 -DEBLEND_MINCONT 0.005 -CLEAN Y -CLEAN_PARAM 1.0 -BACKPHOTO_THICK 24.0 -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_IMAGE /Users/inchiu/GitHub/ComEst/example/DEEPi.cutout.wgt.fits    -FILTER_NAME /usr/local/lib/python2.7/site-packages/comest/templates/filters/default.conv    -STARNNW_NAME /usr/local/lib/python2.7/site-packages/comest/templates/filters/default.nnw
# bnb_sex_args: -DETECT_MINAREA 350 -DETECT_THRESH 1.5 -ANALYSIS_THRESH 1.5 -DEBLEND_NTHRESH 32 -DEBLEND_MINCONT 0.005 -CLEAN Y -CLEAN_

In [3]:
#####
#
# Run SE to detect all sources
#
#####

# run and it will create a bunch of fits containing full srcs
fits_image_example.RunSEforAll()

# create the srcfree fits image and save it.
fits_image_example.SaveSrcFreeFits()

# see what output files you have
print
print "#", "ls path2outdir:"
print
os.listdir(path2outdir)

# run_me:

/usr/local/bin/sex    /Users/inchiu/GitHub/ComEst/example/DEEPi.cutout.fits[0]      -c /usr/local/lib/python2.7/site-packages/comest/templates/sex.config     -PARAMETERS_NAME /usr/local/lib/python2.7/site-packages/comest/templates/sex.params     -CHECKIMAGE_TYPE IDENTICAL,BACKGROUND,BACKGROUND_RMS,-BACKGROUND,OBJECTS,-OBJECTS,SEGMENTATION,APERTURES     -CHECKIMAGE_NAME /Users/inchiu/GitHub/ComEst/example/outdir_example/full.identical.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.background.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.background_rms.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.-background.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.objects.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.-objects.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.segmentation.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/full.apertures.fits     -CATALOG_NAME    /Users/inchiu/GitHub

['full.-background.fits',
 'full.-objects.fits',
 'full.apertures.fits',
 'full.background.fits',
 'full.background_rms.fits',
 'full.cat.fits',
 'full.identical.fits',
 'full.identical.srcfree.fits',
 'full.objects.fits',
 'full.segmentation.fits']

In [4]:
#####
#
# Run SE to detect BnB sources
#
#####

# run and it will create a bunch of fits containing BnB srcs
fits_image_example.RunSEforBnB()

# create the BnB fits image and save it.
fits_image_example.SaveBnBFits()

# see what output files you have
print
print "#", "ls path2outdir:"
print
os.listdir(path2outdir)

# run_me:

/usr/local/bin/sex    /Users/inchiu/GitHub/ComEst/example/DEEPi.cutout.fits[0]      -c /usr/local/lib/python2.7/site-packages/comest/templates/sex.config     -PARAMETERS_NAME /usr/local/lib/python2.7/site-packages/comest/templates/sex.params     -CHECKIMAGE_TYPE IDENTICAL,BACKGROUND,BACKGROUND_RMS,-BACKGROUND,OBJECTS,-OBJECTS,SEGMENTATION,APERTURES     -CHECKIMAGE_NAME /Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.identical.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.background.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.background_rms.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.-background.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.objects.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.-objects.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.segmentation.fits,/Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.apertures.fits     -CATALOG_NAME    /Users/inchiu/GitHub/ComEst/

['bnb.-background.fits',
 'bnb.-objects.fits',
 'bnb.apertures.fits',
 'bnb.background.fits',
 'bnb.background_rms.fits',
 'bnb.cat.fits',
 'bnb.identical.bnb.fits',
 'bnb.identical.fits',
 'bnb.objects.fits',
 'bnb.segmentation.fits',
 'full.-background.fits',
 'full.-objects.fits',
 'full.apertures.fits',
 'full.background.fits',
 'full.background_rms.fits',
 'full.cat.fits',
 'full.identical.fits',
 'full.identical.srcfree.fits',
 'full.objects.fits',
 'full.segmentation.fits']

In [5]:
# ---
# setting the arguments, and we pass them into simulation
# ---
# This is the argument that will be passed into all the simulator.

# use utility function to create argment passer
args_pssr   =   comest.utils.argpasser(
                stamp_size_arcsec  = 20.0,
                mag_dict           = {"lo":MAG_LO, "hi":MAG_HI },
                hlr_dict           = {"lo":0.35, "hi":0.75 },
                fbulge_dict        = {"lo":0.5 , "hi":0.9  },
                q_dict             = {"lo":0.4 , "hi":1.0  },
                pos_ang_dict       = {"lo":0.0 , "hi":180.0},
                ngals_arcmin2      = 15.0,
                nsimimages         = nsimimages,
                ncpu               = ncpu,
        )

# print it out
print
print "#", "args_pssr:"
args_pssr.i_am()
print


# args_pssr:
# stamp_size_arcsec: 20.0
# mag_dict: {'lo': 20.0, 'hi': 25.0}
# hlr_dict: {'lo': 0.35, 'hi': 0.75}
# fbulge_dict: {'lo': 0.5, 'hi': 0.9}
# q_dict: {'lo': 0.4, 'hi': 1.0}
# pos_ang_dict: {'lo': 0.0, 'hi': 180.0}
# ngals_arcmin2: 15.0
# nsimimages: 5
# ncpu: 3



In [6]:
#####
#
# use_modgal 
#
#####

if    use_modgal:
    # ---
    # Anaylsis SE catalog
    # ---
    # Remember the SE output catalog is associating to [full_root_name].
    # Specifically, the filename of the SE output catalog is os.path.join(path2outdir, full_root_name + ".cat.fits")
    
    for nsnset in xrange(nset):
        comest.AnalysisSEcat(path2secat          = os.path.join(path2outdir, full_root_name + ".cat.fits"),
                             img_pixel_scale     = img_pixel_scale,
                             class_star_cut      = 0.8,
                             mag_edges           = np.arange(18.0,28.0,0.05),
                             powerlaw_mag_dict   = {"lo":20.0, "hi":22.0 },
                             mag_dict            = {"lo":MAG_LO, "hi":MAG_HI },
                             hlr_dict            = {"lo":0.35, "hi":1.5  },
                             fbulge_dict         = {"lo":0.5 , "hi":0.9  },
                             q_dict              = {"lo":0.4 , "hi":1.0  },
                             pos_ang_dict        = {"lo":0.0 , "hi":180.0},
                             nsimimages          = nsimimages,
                             random_seed         = int( np.random.random() * 1000* 2.0**(nsnset + np.random.random()) ),
                             path2outdir         = path2outdir,
                             sims_nameroot       = "modgal_" + "%i" % nsnset,
                             saveplot            = True)
    # ---
    # put srcs on the images
    # ---
    # use sims catalog generated from SE catalog to create sims image.
    for nsnset in xrange(nset):
        mef_buldisk = fits_image_example.ModelGalLocator(
                         path2image = fits_image_example.path2outdir + "/" + bnb_root_name + ".identical.bnb.fits",
                         path2readincat = os.path.join(path2outdir, "modgal_" + "%i" % nsnset + "_sims", "modgal_" + "%i" % nsnset + ".sims.cat.fits"),
                         psf_dict   = {"moffat":{ "beta": 3.5, "fwhm": img_fwhm} },
                         ncpu       = ncpu,
                         sims_nameroot = "modgal_" + "%i" % nsnset)
    

    # ---
    # Run SE
    # ---
    for nsnset in xrange(nset):
        fits_image_example.RunSEforSims( sims_nameroot = "modgal_" + "%i" % nsnset, sims_sex_args = fits_image_example.full_sex_args, outputcheckimage = False, tol_fwhm=1.0, path2maskmap = fits_image_example.path2outdir + "/" + bnb_root_name + ".segmentation.fits", ztol = 1.5)

In [7]:
#####
#
# use_bulgal 
#
#####

if    use_bulgal:


    # ---
    # put srcs on the images
    # ---
    for nsnset in xrange(nset):
        mef_buldisk, cat_buldisk = fits_image_example.BulDiskLocator(
                               path2image = fits_image_example.path2outdir + "/" + bnb_root_name + ".identical.bnb.fits",
                               psf_dict   = {"moffat":{ "beta": 3.5, "fwhm": img_fwhm} },
                               random_seed= int( np.random.random() * 1000* 2.0**(nsnset + np.random.random()) ),
                               sims_nameroot = "buldisk_"+"%i" % nsnset,
                               args_pssr = args_pssr)

    # ---
    # Run SE
    # ---
    for nsnset in xrange(nset):
        fits_image_example.RunSEforSims( sims_nameroot = "buldisk_"+"%i" % nsnset, sims_sex_args = fits_image_example.full_sex_args, outputcheckimage = False, tol_fwhm=1.0, path2maskmap = fits_image_example.path2outdir + "/" + bnb_root_name + ".segmentation.fits", ztol = 1.5, ncpu = 3)


# Using BulDiskLocator to put galaxies (bulge + disk) on the targeted image...
# path2image: /Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.identical.bnb.fits
# psf_dict: {'moffat': {'beta': 3.5, 'fwhm': 0.9}}
# random_seed: 949
# sims_nameroot: buldisk_0
# stamp_size_arcsec: 20.0
# mag_dict: {'lo': 20.0, 'hi': 25.0}
# hlr_dict: {'lo': 0.35, 'hi': 0.75}
# fbulge_dict: {'lo': 0.5, 'hi': 0.9}
# q_dict: {'lo': 0.4, 'hi': 1.0}
# pos_ang_dict: {'lo': 0.0, 'hi': 180.0}
# ngals_arcmin2: 15.0
# nsimimages: 5
# ncpu: 3

# Process-1: for file simulated 0 image was done.
# Process-2: for file simulated 1 image was done.
# Process-3: for file simulated 2 image was done.
# Process-2: for file simulated 4 image was done.
# Process-1: for file simulated 3 image was done.

# Total time takes: 4.84720206261


# sims image: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.fits
# sims cat: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0

/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1711: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2001 but corresponding boolean dimension is 2000
  x2 = x_coord_mesh[ i_am_masked ],
/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1712: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2001 but corresponding boolean dimension is 2000
  y2 = y_coord_mesh[ i_am_masked ],
/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1717: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2001 but corresponding boolean dimension is 2000
  x2 = x_coord_mesh[ i_am_masked ],
/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1718: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 2001 but corresponding boolean dimension is 2000
  y2 = y_coord_mesh[ i_am_masked ],



# Read in SE output cat is done.

# Start matching.


# Matching is done.

# Outputing.


# Output cats:
# merged true catalog [all true]: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.sex.merged_true.cat.fits
# matched pairs [matched true] + [matched se]: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.sex.matched_pairs.cat.fits
# unmatched src [true] - [matched true]: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.sex.unmatched.cat.fits
# ghost [se] - [matched se]: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.sex.ghost.cat.fits


# Running SE for sims...
# sims_nameroot: buldisk_1 [Important: the sims_nameroot has to match the nameroot you create sims.]
# sims_sex_args: -DETECT_MINAREA 5   -DETECT_THRESH 1.5 -ANALYSIS_THRESH 1.5 -DEBLEND_NTHRESH 32 -DEBLEND_MINCONT 0.005 -CLEAN Y -CLEAN_PARAM 1.0 -BACKPHOTO_THICK 24.0 -WEIGHT_TYPE MAP_WEIGHT -WEIG

In [15]:
# If it runs successfully, then you should see the directories with names `buldisk_?_sims` are created.
# Several catalogs are outputed.
# Lets take a look.
print
print "#", "ls path2outdir:"
print
os.listdir(path2outdir + "/buldisk_0_sims")

# *.sims.fits is the simulated image.
# *.sims.cat.fits is the input catalog.
# ComEst will run Sextractor automatically in the last step.
# *.sims.sex.?.cat.fits is the output catalog returned by Sextractor.
# Then, ComEst matches the Sextractor output catalogs to the input catalog and 
# produce a bunch of stacked catalogs. See Table 1 of arxiv 1605.02650
# *.sims.sex.ghost.cat.fits
# *.sims.sex.matched_pairs.cat.fits
# *.sims.sex.merged_true.cat.fits
# *.sims.sex.unmatched.cat.fits




# ls path2outdir:



['buldisk_0.sims.cat.fits',
 'buldisk_0.sims.fits',
 'buldisk_0.sims.sex.0.cat.fits',
 'buldisk_0.sims.sex.1.cat.fits',
 'buldisk_0.sims.sex.2.cat.fits',
 'buldisk_0.sims.sex.3.cat.fits',
 'buldisk_0.sims.sex.4.cat.fits',
 'buldisk_0.sims.sex.ghost.cat.fits',
 'buldisk_0.sims.sex.matched_pairs.cat.fits',
 'buldisk_0.sims.sex.merged_true.cat.fits',
 'buldisk_0.sims.sex.unmatched.cat.fits']

In [8]:
# Same for real galaxies and point sources. Just change the simulators.

#####
#
# use_reagal 
#
#####


if    use_reagal:

    # ---
    # put srcs on the images
    # ---
    for nsnset in xrange(nset):
        mef_realgal, cat_realgal = fits_image_example.RealGalLocator(
                               path2image = fits_image_example.path2outdir + "/" + bnb_root_name + ".identical.bnb.fits",
                               psf_dict   = {"moffat":{ "beta": 3.5, "fwhm": img_fwhm} },
                               random_seed= int( np.random.random() * 1000* 2.0**(nsnset + np.random.random()) ),
                               sims_nameroot = "realgal_"+"%i" % nsnset,
                               args_pssr = args_pssr)
    # ---
    # Run SE
    # ---
    for nsnset in xrange(nset):
        fits_image_example.RunSEforSims( sims_nameroot = "realgal_"+"%i" % nsnset, sims_sex_args = fits_image_example.full_sex_args, outputcheckimage = False, tol_fwhm=1.0, path2maskmap = fits_image_example.path2outdir + "/" + bnb_root_name + ".segmentation.fits", ztol = 1.5, ncpu = 3)


#####
#
# use_pntsrc 
#
#####


if    use_pntsrc:
    # ---
    # put srcs on the images
    # ---
    for nsnset in xrange(nset):
        mef_pntsrcs, cat_pntsrcs = fits_image_example.PntSrcLocator(
                               path2image = fits_image_example.path2outdir + "/" + bnb_root_name + ".identical.bnb.fits",
                               psf_dict   = {"moffat":{ "beta": 3.5, "fwhm": img_fwhm} },
                               random_seed= int( np.random.random() * 1000* 2.0**(nsnset + np.random.random()) ),
                               sims_nameroot = "pntsrc_"+"%i" % nsnset,
                               args_pssr = args_pssr)
    # ---
    # Run SE
    # ---
    for nsnset in xrange(nset):
        fits_image_example.RunSEforSims( sims_nameroot = "pntsrc_" +"%i" % nsnset, sims_sex_args = fits_image_example.full_sex_args, outputcheckimage = False, tol_fwhm=1.0, path2maskmap = fits_image_example.path2outdir + "/" + bnb_root_name + ".segmentation.fits", ztol = 1.5, ncpu = 3)




# Using RealGalLocator to put galaxies (bulge + disk) on the targeted image...
# path2image: /Users/inchiu/GitHub/ComEst/example/outdir_example/bnb.identical.bnb.fits
# psf_dict: {'moffat': {'beta': 3.5, 'fwhm': 0.9}}
# random_seed: 926
# sims_nameroot: realgal_0
# stamp_size_arcsec: 20.0
# mag_dict: {'lo': 20.0, 'hi': 25.0}
# hlr_dict: {'lo': 0.35, 'hi': 0.75}
# fbulge_dict: {'lo': 0.5, 'hi': 0.9}
# q_dict: {'lo': 0.4, 'hi': 1.0}
# pos_ang_dict: {'lo': 0.0, 'hi': 180.0}
# ngals_arcmin2: 15.0
# nsimimages: 5
# ncpu: 3

# Process-14: for file simulated 1 image was done.
# Process-13: for file simulated 0 image was done.
# Process-15: for file simulated 2 image was done.
# Process-13: for file simulated 4 image was done.
# Process-14: for file simulated 3 image was done.

# Total time takes: 878.95004797


# sims image: /Users/inchiu/GitHub/ComEst/example/outdir_example/realgal_0_sims/realgal_0.sims.fits
# sims cat: /Users/inchiu/GitHub/ComEst/example/outdir_example/realgal_0_sims/realg

In [9]:
#####
#
# Histgram
#
#####

# Last, we extract the completeness and purity from the stacked catalogs in each subdirectory.
# In general, you can save the results in any way you want, you do not have to save it into dict like what I did.

HIST_DICT        =       {
    ttype + "_" + "%i" % nsnset   :   {
        "com"   :   fits_image_example.DeriveCom(sims_nameroot = ttype + "_" + "%i" % nsnset, x_steps_arcmin = 3.0, y_steps_arcmin = 3.0, mag_edges = np.arange(20.0,27.0,0.1), save_files = True),
        "pur"   :   fits_image_example.DerivePur(sims_nameroot = ttype + "_" + "%i" % nsnset, x_steps_arcmin = 3.0, y_steps_arcmin = 3.0, mag_edges = np.arange(20.0,27.0,0.1), save_files = True),
    } for ttype in np.array(["buldisk", "realgal", "pntsrc"]) for nsnset in xrange(nset)
}


# Save it into pickle file.

# Save the pickle file
FF      =       open(os.path.join(path2outdir, "output.pickle"), "wb")
pickle.dump(HIST_DICT, FF)
FF.close()

# Then it is done!
print
print "#", "Done! Check the output.pickle file."
print


# sims_nameroot: buldisk_0
# x_steps: 3.0 [arcmin]
# y_steps: 3.0 [arcmin]
# mag_edges: [ 20.   20.1  20.2  20.3  20.4  20.5  20.6  20.7  20.8  20.9  21.   21.1
  21.2  21.3  21.4  21.5  21.6  21.7  21.8  21.9  22.   22.1  22.2  22.3
  22.4  22.5  22.6  22.7  22.8  22.9  23.   23.1  23.2  23.3  23.4  23.5
  23.6  23.7  23.8  23.9  24.   24.1  24.2  24.3  24.4  24.5  24.6  24.7
  24.8  24.9  25.   25.1  25.2  25.3  25.4  25.5  25.6  25.7  25.8  25.9
  26.   26.1  26.2  26.3  26.4  26.5  26.6  26.7  26.8  26.9]



/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1976: RuntimeWarning: invalid value encountered in divide
  fcom_of_mag                 =   matched_paris_dict["hist1d"] * 1.0 / true_dict["hist1d"]
/usr/local/lib/python2.7/site-packages/comest/ComEst.py:1980: RuntimeWarning: invalid value encountered in divide
  fcom_of_xy                  =   matched_paris_dict["hist2d"] * 1.0 / true_dict["hist2d"]



# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.fcommag.dat
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.fcomxy.fits
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.nsrc_merged_true_xy.fits
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.nsrc_matched_pairs_xy.fits


# sims_nameroot: buldisk_0
# x_steps: 3.0 [arcmin]
# y_steps: 3.0 [arcmin]
# mag_edges: [ 20.   20.1  20.2  20.3  20.4  20.5  20.6  20.7  20.8  20.9  21.   21.1
  21.2  21.3  21.4  21.5  21.6  21.7  21.8  21.9  22.   22.1  22.2  22.3
  22.4  22.5  22.6  22.7  22.8  22.9  23.   23.1  23.2  23.3  23.4  23.5
  23.6  23.7  23.8  23.9  24.   24.1  24.2  24.3  24.4  24.5  24.6  24.7
  24.8  24.9  25.   25.1  25.2  25.3  25.4  25.5  25.6  25.7  25.8  25.9
  26.   26.1  26.2  26.3  26.4  26.5  26.6  26.7  26.8  26.9]



/usr/local/lib/python2.7/site-packages/comest/ComEst.py:2191: RuntimeWarning: invalid value encountered in divide
  fpur_of_mag                 =   matched_pairs_dict["hist1d"] * 1.0 / ( matched_pairs_dict["hist1d"] + ghost_dict["hist1d"] )
/usr/local/lib/python2.7/site-packages/comest/ComEst.py:2196: RuntimeWarning: invalid value encountered in divide
  fpur_of_xy                  =   matched_pairs_dict["hist2d"] * 1.0 / ( matched_pairs_dict["hist2d"] + ghost_dict["hist2d"] )



# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.fpurmag.dat
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.fpurxy.fits
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_0_sims/buldisk_0.sims.nsrc_allse_xy.fits


# sims_nameroot: buldisk_1
# x_steps: 3.0 [arcmin]
# y_steps: 3.0 [arcmin]
# mag_edges: [ 20.   20.1  20.2  20.3  20.4  20.5  20.6  20.7  20.8  20.9  21.   21.1
  21.2  21.3  21.4  21.5  21.6  21.7  21.8  21.9  22.   22.1  22.2  22.3
  22.4  22.5  22.6  22.7  22.8  22.9  23.   23.1  23.2  23.3  23.4  23.5
  23.6  23.7  23.8  23.9  24.   24.1  24.2  24.3  24.4  24.5  24.6  24.7
  24.8  24.9  25.   25.1  25.2  25.3  25.4  25.5  25.6  25.7  25.8  25.9
  26.   26.1  26.2  26.3  26.4  26.5  26.6  26.7  26.8  26.9]


# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/buldisk_1_sims/buldisk_1.sims.fcommag.dat
# output: /Users/inchiu/GitHub/ComEst/example/outdir_example/